## Installing Important Packages

In [ ]:
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install jiwer
!pip install kenlm
!pip install pyctcdecode

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


# Section 1: Automatic Speech Recognition

## 1.1 Import important libraries

In [ ]:
from datasets import Dataset, Audio
from transformers import pipeline
import pandas as pd
import evaluate
import torch
import time

## 1.2 Load Urdu Recipes dataset

In [ ]:
df = pd.read_excel('new-urdu-recipes.xlsx')
df.head()

,Id,recipe_name,additional_phrase,ing_name,Ing_quantity,Ing_unit,source,audio
0,1,بلوچی گوشت,بلوچی گوشت بنانے کے اجزاء,NaN,NaN,NaN,kfoods,dataset\Balochi Gosht - kfoods.wav
1,1,بلوچی گوشت,NaN,بون لیس مٹن,½,کلو,kfoods,dataset\Balochi Gosht - kfoods.wav
2,1,بلوچی گوشت,NaN,ادرک لہسن کا پیسٹ,1,کھانے کا چمچہ,kfoods,dataset\Balochi Gosht - kfoods.wav
3,1,بلوچی گوشت,NaN,ثابت لال مرچ,6,عدد,kfoods,dataset\Balochi Gosht - kfoods.wav
4,1,بلوچی گوشت,NaN,ہری مرچ,3,عدد,kfoods,dataset\Balochi Gosht - kfoods.wav


## 1.3 Preprocessing dataset

In [ ]:
df['Ing_quantity'] = df['Ing_quantity'].replace(r' ', '', regex= True)
df['ing_name'] = df['ing_name'].str.strip('\t')
df = df.fillna('').astype(str)
df = df.replace(r'[\n\xa0\u200c]', '', regex = True)

### Converting numeric quantities into words

In [ ]:
urdu_counting = {'0.25': 'ایک چوتھائی', '0.5': 'آدھا', '½-1': 'ڈیڑھ','1½': 'ڈیڑھ', '½-2': 'ڈھائی',
 '2½': 'ڈھائی', '½': 'آدھا', '1': 'ایک', '2': 'دو', '3': 'تین', '4': 'چار', '5': 'پانچ', '6': 'چھ',
 '7': 'سات', '8': 'آٹھ', '9': 'نو', '10': 'دس', '11': 'گیارہ', '12': 'بارہ', '13': 'تیرہ', '14': 'چودہ',
 '15': 'پندرہ', '16': 'سولہ', '17': 'سترہ', '18': 'اٹھارہ', '19': 'انیس', '20': 'بیس', '21': 'اکیس', '22': 'بائیس',
 '23': 'تئیس', '24': 'چوبیس', '25': 'پچیس', '26': 'چھببیس', '27': 'ستائیس', '28': 'اٹھائیس', '29': 'انتیس', '30': 'تیس',
 '31': 'اکتیس', '32': 'بتیس', '33': 'تینتیس', '34': 'چونتیس', '35': 'پینتیس', '36': 'چھتیس', '37': 'سینتیس', '38': 'اڑتیس',
 '39': 'انتالیس', '40': 'چالیس', '41': 'اکتالیس', '42': 'بیالیس', '43': 'تینتالیس', '44': 'چوالیس', '45': 'پینتالیس', '46': 'چھیالیس',
 '47': 'سینتالیس', '48': 'اڑتالیس', '49': 'انچاس', '50': 'پچاس', '51': 'اکیاون', '52': 'باون', '53': 'ترپن', '54': 'چون', 
 '55': 'پچپن', '56': 'چھپن', '57': 'ستاون', '58': 'اٹھاون', '59': 'انسٹھ', '60': 'ساٹھ', '61': 'اکسٹھ', '62': 'باسٹھ',
 '63': 'ترسٹھ', '64': 'چوسٹھ', '65': 'پینسٹھ', '66': 'چھیاسٹھ', '67': 'سڑسٹھ', '68': 'اٹھسٹھ', '69': 'انہتر', '70': 'ستر',
 '71': 'اکہتر', '72': 'بہتر', '73': 'تہتر', '74': 'چوہتر', '75': 'پچہتر', '76': 'چھہتر', '77': 'ستتر', '78': 'اٹھہتر',
 '79': 'اناسی', '80': 'اسی', '81': 'اکیاسی', '82': 'بیاسی', '83': 'تراسی', '84': 'چوراسی', '85': 'پچاسی', '86': 'چھیاسی',
 '87': 'ستاسی', '88': 'اٹھاسی', '89': 'نواسی', '90': 'نوے', '91': 'اکانوے', '92': 'بانوے', '93': 'ترانوے', '94': 'چورانوے',
 '95': 'پچانوے', '96': 'چھیانوے', '97': 'ستانوے', '98': 'اٹھانوے', '99': 'ننانوے', '100': 'سو', '150': 'ایک سو پچاس', '200': 'دو سو',
 '250': 'دو سو پچاس', '300': 'تین سو', '350': 'تین سو پچاس', '400': 'چار سو',
 '450': 'چار سو پچاس', '500': 'پانچ سو'}

In [ ]:
def num_to_words(value): 
    if '-' in value:
        tmp = value.split('-')
        if tmp[0] != '½':
            tmp[0] = urdu_counting[tmp[0]]
            tmp[1] = urdu_counting[tmp[1]]
            return (' سے '.join(tmp))           
    
    if value in urdu_counting.keys():
        return urdu_counting[value]
    return  value
for i in df.index:
    df.loc[i, 'Ing_quantity'] = num_to_words(df.loc[i, 'Ing_quantity'])
    df.loc[i, 'ing_name'] = df.loc[i, 'ing_name'].replace('14', num_to_words('14'))
    df.loc[i, 'ing_name'] = df.loc[i, 'ing_name'].replace('16', num_to_words('16'))

### Combine necessary columns to create one complete Ingredient Phrase for each recipe

In [ ]:
df['text'] = df[['additional_phrase', 'ing_name', 'Ing_quantity', 'Ing_unit']].astype(str).apply(' '.join, axis=1)
chars_to_remove_regex = r'[\ ِ\'ُ\،\?\.\!\-\;\:\"\“\%\‘\”\�\'\)\(]| +\n'
df['text'] = df['text'].replace(chars_to_remove_regex, ' ', regex = True).replace(r' +', ' ', regex = True)
df['audio'] = df["audio"].replace(r"\\", "/", regex = True)
df.head()

,Id,recipe_name,additional_phrase,ing_name,Ing_quantity,Ing_unit,source,audio,text
0,1,بلوچی گوشت,بلوچی گوشت بنانے کے اجزاء,,,,kfoods,dataset/Balochi Gosht - kfoods.wav,بلوچی گوشت بنانے کے اجزاء
1,1,بلوچی گوشت,,بون لیس مٹن,آدھا,کلو,kfoods,dataset/Balochi Gosht - kfoods.wav,بون لیس مٹن آدھا کلو
2,1,بلوچی گوشت,,ادرک لہسن کا پیسٹ,ایک,کھانے کا چمچہ,kfoods,dataset/Balochi Gosht - kfoods.wav,ادرک لہسن کا پیسٹ ایک کھانے کا چمچہ
3,1,بلوچی گوشت,,ثابت لال مرچ,چھ,عدد,kfoods,dataset/Balochi Gosht - kfoods.wav,ثابت لال مرچ چھ عدد
4,1,بلوچی گوشت,,ہری مرچ,تین,عدد,kfoods,dataset/Balochi Gosht - kfoods.wav,ہری مرچ تین عدد


In [ ]:
new_df = df.groupby(['Id', 'recipe_name', 'audio'])['text'].apply(' '.join).reset_index()

In [ ]:
new_df.head()

,Id,recipe_name,audio,text
0,1,بلوچی گوشت,dataset/Balochi Gosht - kfoods.wav,بلوچی گوشت بنانے کے اجزاء بون لیس مٹن آدھا ک...
1,10,فروٹی اسموتھی,dataset/Fruity Smoothie - kfoods.wav,فروٹی اسموتھی بنانے کے اجزاء آڑو ایک کپ در...
2,11,کرسپ اسپنچ وِد سیلیڈ,dataset/Crisp Chicken with Salad - kfoods.wav,کرسپ اسپنچ و د سیلیڈ بنانے کے اجزاء چکن بریس...
3,12,کریم آلو,dataset/Cream Aloo - kfoods.wav,کریم آلو بنانے کے اجزاء آلو ایک کلو ہری پیا...
4,13,انڈے کی بریانی,dataset/Anday ki biryani.wav,انڈے کی بریانی بنانے کے اجزاء ابلے انڈے آٹھ ...


## 1.4 Convert the audio recordings into float arrays for model input using 🤗 Datasets

In [ ]:
dataset = Dataset.from_pandas(new_df)
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
sampling_rate = dataset.features["audio"].sampling_rate

In [ ]:
dataset

Dataset({
    features: ['Id', 'recipe_name', 'audio', 'text'],
    num_rows: 78
})

## 1.5 Generate audio transcriptions using 🤗 pre-trained Automatic Speech Recognition models and 🤗 Pipelines

In [ ]:
audios = []
for i in range (dataset.num_rows):
  audios.append(dataset[i]['audio']['array'])

### Using Whisper fine-tuned model on urdu from publicly available models 

In [ ]:
asr_whisper = pipeline("automatic-speech-recognition", model="ihanif/whisper-medium-urdu", chunk_length_s=30, device = "cuda:0", batch_size=2)

In [ ]:
start = time.time()

whisper = asr_whisper(audios)

end = time.time()

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1070: U

Time taken for whisper model: 1326.9922063350677 seconds


In [ ]:
print(f'Time taken for whisper model: {end - start} seconds')

### Using Wav2Vec2 fine-tuned model on urdu from publicly available models 

In [ ]:
asr_wav2vec2 = pipeline("automatic-speech-recognition", model="anuragshas/wav2vec2-xls-r-300m-ur-cv9-with-lm", chunk_length_s=30, device = "cuda:0", batch_size=2)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
start = time.time()

wav2vec2 = asr_wav2vec2(audios)

end = time.time()

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentiall

Time taken for wav2vec2 model: 92.67782926559448 seconds


In [ ]:
print(f'Time taken for wav2vec2 model: {end - start} seconds')

### Display generated Audio Transcriptions

In [ ]:
recipe_names = new_df['recipe_name'].values.tolist()
gold_references = new_df['text'].values.tolist()

In [ ]:
predictions_df = pd.DataFrame({'Recipe Name': recipe_names, 'Original Text': gold_references, 'Whisper Predicted Text': whisper, 'Wav2Vec2 Predicted Text': wav2vec2})

In [ ]:
predictions_df['Whisper Predicted Text'] = predictions_df["Whisper Predicted Text"].replace(r'،', '', regex = True)
predictions_df['Wav2Vec2 Predicted Text'] = predictions_df["Wav2Vec2 Predicted Text"].replace(r'،', '', regex = True)

In [ ]:
predictions_df.head()

,Recipe Name,Original Text,Whisper Predicted Text,Wav2Vec2 Predicted Text
0,بلوچی گوشت,بلوچی گوشت بنانے کے اجزاء بون لیس مٹن آدھا ک...,بلوچی گوشت بنانے کے چمچے، تیل تین کھانے کے چمچ...,بلوچی گوشت بنانے کے سا مول لیس مٹ دھکیل ادرک ل...
1,فروٹی اسموتھی,فروٹی اسموتھی بنانے کے اجزاء آڑو ایک کپ در...,فروٹی سمودی بنانے کے اتزا اڑو ایک کپ ترمیانے گ...,فروٹی اس مودی بنانے کے عجز اڑو ایک کپ درمیانے ...
2,کرسپ اسپنچ وِد سیلیڈ,کرسپ اسپنچ و د سیلیڈ بنانے کے اجزاء چکن بریس...,کرسپ سپنچ ویٹ سالٹ بنانے کے اجزاء چکن بریسٹ دو...,کرسپسپنچوت سالٹ بنانے کے از چکن برس دو عدد کھی...
3,کریم آلو,کریم آلو بنانے کے اجزاء آلو ایک کلو ہری پیا...,کریم آلو بنانے کے عدزہ آلو آدھا کلو ہری پیاز د...,کریم آلو بنانے کے عصا آلو ادا کیلوں ہری پیاز د...
4,انڈے کی بریانی,انڈے کی بریانی بنانے کے اجزاء ابلے انڈے آٹھ ...,انڈے کی پیانی، گھیتزا، ابلے انڈے آٹ اتد، چاول...,انڈے کی بیانی کہ عدزاعبلےانڈے اٹھتر چاول دو پا...


In [ ]:
predictions_df.to_csv('Speech2TextPredictions.csv')

## 1.6 Evaluating Word Error Rate (WER) for wav2vec2 and whisper predictions

In [ ]:
cer = evaluate.load('cer')
wer = evaluate.load('wer')

In [ ]:
whisper_preds = predictions_df['Whisper Predicted Text'].values.tolist()
wav2vec2_preds = predictions_df['Wav2Vec2 Predicted Text'].values.tolist()

In [ ]:
wer_whisper = 100* wer.compute(predictions=whisper_preds, references = gold_references)
cer_whisper = 100* cer.compute(predictions=whisper_preds, references = gold_references)
wer_whisper, cer_whisper

(52.94006784771956, 34.996037869625056)

In [ ]:
wer_wav2vec2 = 100* wer.compute(predictions=wav2vec2_preds, references = gold_references)
cer_wav2vec2  = 100* cer.compute(predictions=wav2vec2_preds, references = gold_references)
wer_wav2vec2 , cer_wav2vec2 

(59.950998869204675, 24.669474913458732)